## Computational Physics 2 (WS23/24) – Warm-up exercise

**Deadline: 31.10.2023 at 23:59**

Group: *Henri Poincaré *
Students: *Max Heimann 580560, Lyding (Lukas) Brumm 604522, Burak Varol 623941*

You will implement and test two algorithms: **conjugate gradient** and **power method**. We will see in a moment what they are useful for. Fill the notebook following the instructions.

### Initialization

Here we load the needed libraries and we initialize the random number generator. **Important**: when using a random number generator, the seed needs to be set only once, preferebly at the beginning of the program.

In [1]:
import numpy as np

rng = np.random.Generator(np.random.PCG64(12345))

### Positive-definite matrices

Both algorithms will deal with hermitian positive-definite matrices. Recall:

- Given a complex square matrix $A$, its hermitian conjugate $A^\dagger$ is defined as its transposed complex-conjugated, i.e. $(A^\dagger)_{ij} = (A_{ji})^*$.
- A complex square matrix $A$ is said to be hermitian if $A=A^\dagger$.
- An hermitian matrix $A$ is said to be positive-definite if all its eigenvalues are positive.

The following function generates and returns a random positive-definite matrix, along with its eigenvactors and eigenvalues.

In [2]:
# The function 'generate_positive_definite_matrix' contructs an NxN positive-definite matrix 'A',
# its matrix of eigenvectors and its eigenvalues.
#
# Input parameters:
#    N (integer)        : size of output matrix 'A'
#    kappa (double)     : condition number of the output matrix 'A'
#                         see https://en.wikipedia.org/wiki/Condition_number#Matrices
# Output values: (A, U, evalues)
#    A (np.matrix)      : positive-definite NxN matrix with condition number kappa
#    U (np.matrix)      : NxN unitary matrix; each column of 'U' is an eigenvector of 'A'
#    evalues (np.array) : N-component array with eigenvalues of 'A'

def generate_positive_definite_matrix(N,kappa=10.):
    assert isinstance(N, int) and N > 1 , "N=" + str(N) + " must be an integer >= 2"
    assert isinstance(kappa, float) and kappa > 0. , "kappa=" + str(kappa) + " must be a positive float"
    
    rmat = np.asmatrix(rng.standard_normal(size=(N,N)) + 1j * rng.standard_normal(size=(N,N)))
    U , _ = np.linalg.qr(rmat,mode='complete')
    evalues = np.concatenate((1. + kappa*rng.random(N-2),[1.,kappa]))
    D = np.asmatrix(np.diag(evalues))
    A = np.matmul(np.matmul(U,D),U.getH())
    
    return A, U , evalues

### Power method

Given a positive-definite matrix $A$, the power method allows to approximate its largest eigenvalue and the corresponding eigenvector with a certain specified tolerance $\epsilon$. It is an iterative method: a number of steps are repeated cyclically, at each iteration one gets a better approximation of the eigenvalue and eigenvectors, the iteration is stopped when the approximation is good enough. It works as follows:

1. Generate a random complex vector $v$ with norm equal to 1.
2. Calculate $w=Av$ and $\mu = \| w \|$.
3. If $\| w - \mu v \| < \epsilon$, stop iteration and returns $\mu$ and $v$ are eigenvalue and eigenvector.
4. Replace $v \leftarrow \mu^{-1} w$ and repeat from 2.

**Task:** Implement the power method within the function ```power_method```, with the following specifications.

The *vector* $v$ is not necessarily a one-dimensional array, we want the flexibility to use more abstract vector spaces whose elements are generic $d$-dimensional arrays. In practice, the *vectors* $v$ must be implemented as ```numpy.ndarrays```. In this setup, the squared norm $\|v\|^2$ of the *vector* $v$ is given by the sum of the squared absolute value of all elements of $v$. Moreover, the *matrix* $A$ really needs to be thought as a linear function acting on the elements of the abstract vector space.

```power_method``` must be a function that takes three inputs:
- ```vshape``` is the shape of the elements $v$ of the abstract vector space;
- ```apply_A``` is a function that takes the vector $v$ (represented as an instance of ```numpy.ndarrays```) and returns the vector $Av$ (represented as an instance of ```numpy.ndarrays``` with the same shape as $v$);
- ```epsilon``` is the tolerance.

```power_method``` must return:
- the largest eignevalue $\mu$;
- the corresponding eigenvector (represented as an instance of ```numpy.ndarrays``` with the same shape as the input of ```apply_A```);
- the number of iterations.

A test function is provided below. Your implementation of ```power_method``` needs to pass this test.

In [3]:
# The function 'power_method' calculates an approximation of the largest eigenvalue 'mu'
# and corresponding eigenvector 'v' of the positive-definite linear map 'A'. The quality
# of the approximation is dictated by the tolerance 'epsilon', in the sense that the
# approximated eigenvalue and eigenvector satisfy
#   | A(v) - mu*v | < epsilon
#
# The vectors over which 'A' acts are generically d-dimensional arrays. More precisely,
# they are instances of 'numpy.ndarray' with shape 'vshape'.
#
# The linear map 'A' is indirectly provided as a function 'apply_A' which takes a vector
# v and returns the vector A(v).
#
# Input parameters of power_method:
#    vshape (tuple of ints) : shape of the arrays over which 'A' acts
#    apply_A (function)     : function v -> A(v)
#    epsilon (float)        : tolerance
# Output values: (mu, v, niters)
#    mu (float)             : largest eigenvalue of A
#    v (numpy.ndarray)      : corresponding eigenvector
#    niters (int)           : number of iterations

def power_method(vshape,apply_A,epsilon):
    """Calculate the biggest eigenvalue of an Hermitian Operator a 

    Args:
        vshape (tuple of ints) : shape of the arrays over which 'A' acts
        apply_A (function)     : function v -> A(v)
        epsilon (float)        : tolerance

    Returns:
        mu (float)             : largest eigenvalue of A
        v (numpy.ndarray)      : corresponding eigenvector
        niters (int)           : number of iterations
    """
    assert callable(apply_A) , "apply_A must be a function"
    assert isinstance(epsilon, float) and epsilon > 0. , "epsilon=" + str(epsilon) + " must be a positive float"
    assert isinstance(vshape,tuple) , "vshape must be a tuple"
    esqr = epsilon**2
    vinit = np.random.random_sample(vshape)
    norm = np.linalg.norm(vinit) # same as np.sqrt(sum(vinit**2)) but probably faster
    v = vinit/norm
    Av = apply_A(v)
    mu = np.linalg.norm(Av)
    res = Av-v*mu
    niters = 0
    while esqr < np.real(np.vdot(res,res)):
        v = Av/mu
        mu = np.linalg.norm(v)
        Av = apply_A(v)
        res = Av-v*mu
        niters+=1
    v=v/mu
    return mu, v, niters

#### Test

Run the following cell. If the power method is correctly implemented, then the test will pass.

In [4]:
def test_power_method():

    def test_engine(shape,epsilon):
        
        N = int(np.prod(shape))
        A , _ , _ = generate_positive_definite_matrix(N)
        
        def apply_A(v):
            assert isinstance(v,np.ndarray) , "v must be an np.ndarray"
            assert v.shape==shape , "v has shape "+str(v.shape)+", it must have shape "+str(shape)
            return np.asarray(np.dot(A,v.flatten())).reshape(shape)
        
        mu , v , niters = power_method(shape,apply_A,epsilon)
        delta = apply_A(v) - mu*v
        res = np.sqrt(np.vdot(delta,delta).real)
        print("shape = " , shape , "\tresidue = " , res , "\titerations = " , niters , "\tTest passes: " , res<=epsilon)
    
    
    test_engine((4,),1.e-8)
    test_engine((1,5),1.e-12)
    test_engine((3,2,4),1.e-8)
    test_engine((5,2),1.e-12)

test_power_method()

shape =  (4,) 	residue =  9.637436543930886e-10 	iterations =  622 	Test passes:  True
shape =  (1, 5) 	residue =  9.955871969088826e-14 	iterations =  1152 	Test passes:  True
shape =  (3, 2, 4) 	residue =  9.20675726347073e-10 	iterations =  663 	Test passes:  True
shape =  (5, 2) 	residue =  9.853133957728112e-14 	iterations =  1642 	Test passes:  True


### Conjugate gradient

**Task.**
1. Read about the conjugate gradient on Wikipedia.
2. Implement the conjugate gradient, using same conventions as for the power method.
3. Write a description of the algorithm here (in the same spirit as the description of the power method).
4. Run and pass the test provided below.
5. Discuss intermediate steps with tutors.

# Conjugate Gradient Algorithm

The Conjugate Gradient (CG) algorithm is an iterative method used for solving systems of linear equations. It's goal is to calculate the solution to the equation $ \bold{A}\vec{x} = \vec{b}$, $\bold{A}^{-1}\vec{b}$.  It's particularly well-suited for symmetric positive-definite matrices, as well as hermitian matricies. 

## Algorithm Steps:

1. **Initialization**:
   - Choose an initial guess for the solution, denoted as $\vec{x}_0$.
   - Initialize the residual $\vec{r}_0 = \bold{A}\vec{x}_0 - \vec{b}$  
   - Set the search direction $\vec{d}_0 = \vec{r}_0$.

2. **Iterate**:
   - For each iteration (k = 0, 1, 2, ...):
      - Compute $\vec{z} = \bold{A}\vec{d}_0$ and to save computations later on
      - Compute the step size $\alpha _k = \frac{\vec{r}^{\dagger}_{k}\vec{r}_{k}}{\vec{d}^{\dagger}_k\vec{z}}$
         - this minimizes the residual in the search direction
      - Update the solution: $\vec{x}_{k+1} = \vec{x}_{k} + \alpha _k \vec{d}_k$.
      - Update the residual: $\vec{r}_{k+1} = \vec{r}_k - \alpha _k \vec{z}$.
      - Compute the next search direction using the new residual: $\beta _k = \frac{\vec{r}^{\dagger}_{k}\vec{r}_{k}}{\vec{r}^{\dagger}_{k+1}\vec{r}_{k+1}}$, $\vec{d}_{k+1} = \vec{r}_{k+1} + \beta _k \vec{d}_k$.

3. **Termination**:
   - Repeat the iterations until a convergence criterion is met, such as reaching a specified tolerance or a maximum number of iterations.

### Additional Comments on implementation
The code below is not a 1-to-1 implementation of the algorithm above. For example the value of $\vec{r}^{\dagger}_{k}\vec{r}_{k}$ is stored in a variable since it appears in multiple steps.

A second difference is the implementation of the "harder" convergence criterion in a if statement. This is implemented to circumvent the accumulation of floating point error in the residual: $|\vec{r}|$ might be smaller than $\epsilon$, however since it is computed iteratively, accumulating floating point error might occur. To make sure this error won't be higher than the actually wanted precission, the r value is recalculated every 100th loop. 

In [46]:
# The function 'conjugate_gradient' calculates an approximation 'x' of 'A^{-1}(b)', where
# 'A' is a positive-definite linear map 'A', and 'b' is a vector in the domain of 'A'.
# The quality of the approximation is dictated by the tolerance 'epsilon', in the sense
# that the following inequality is satisfied
#   | A(x) - b | <= epsilon |b|
#
# The vectors over which 'A' acts are generically d-dimensional arrays. More precisely,
# they are instances of 'numpy.ndarray' with shape 'vshape'.
#
# The linear map 'A' is indirectly provided as a function 'apply_A' which takes a vector
# v and returns the vector A(v).
#
# Input parameters of power_method:
#    apply_A (function)     : function v -> A(v)
#    b (numpy.ndarray)      : vector 'b'
#    epsilon (float)        : tolerance
# Output values: (x, niters)
#    x (numpy.ndarray)      : approximation of 'A^{-1}(b)'
#    niters (int)           : number of iterations

def conjugate_gradient(apply_A,b,epsilon):
    """Calculate A^-1b with the conjugate gradient method

    Args:
        apply_A (function)     : function v -> A(v)
        b (numpy.ndarray)      : vector 'b'
        epsilon (float)        : tolerance

    Returns:
        x (numpy.ndarray)      : approximation of 'A^{-1}(b)'
        niters (int)           : number of iterations
    """
    assert callable(apply_A) , "apply_A must be a function"
    assert isinstance(epsilon, float) and epsilon > 0. , "epsilon=" + str(epsilon) + " must be a positive float"
    assert isinstance(b,np.ndarray) , f"b={b} must be an np.ndarray"
    x = np.random.random_sample(b.shape)
    r = b - apply_A(x)
    d = r
    niters = 0
    rsqr = np.real(np.vdot(r,r))
    esqr = epsilon**2
    
    while (rsqr > esqr):
        z = apply_A(d)
        a = rsqr/np.real(np.vdot(d,z))
        x = x + a*d
        if niters % 2 == 1:
            r = -apply_A(x) + b 
            d = r
        else:
            r = r-a*z
        rsqr_old = rsqr
        rsqr = np.real(np.vdot(r,r))
        beta = rsqr/rsqr_old
        d = r + beta*d
        niters += 1
    return x, niters

In [47]:
def test_conjugate_gradient():

    def test_engine(shape,epsilon):
        
        N = int(np.prod(shape))
        A , _ , _ = generate_positive_definite_matrix(N)
        b = rng.standard_normal(size=shape) + 1j * rng.standard_normal(size=shape)
        
        def apply_A(v):
            assert isinstance(v,np.ndarray) , "v must be an np.ndarray"
            assert v.shape==shape , "v has shape "+str(v.shape)+", it must have shape "+str(shape)
            return np.asarray(np.dot(A,v.flatten())).reshape(shape)
        
        x , niters = conjugate_gradient(apply_A,b,epsilon)
        delta = apply_A(x) - b
        res = np.sqrt(np.vdot(delta,delta).real)
        print("shape = " , shape , "\tresidue = " , res , "\titerations = " , niters , "\tTest passes: " , res<=epsilon*np.sqrt(np.vdot(b,b).real))
    
    
    test_engine((4,),1.e-8)
    test_engine((1,5),1.e-12)
    test_engine((3,2,4),1.e-8)
    test_engine((5,2),1.e-12)

test_conjugate_gradient()

shape =  (4,) 	residue =  5.440918002156144e-09 	iterations =  52 	Test passes:  True
shape =  (1, 5) 	residue =  7.385550204290278e-13 	iterations =  77 	Test passes:  True
shape =  (3, 2, 4) 	residue =  6.6786408135607885e-09 	iterations =  50 	Test passes:  True
shape =  (5, 2) 	residue =  7.781023332422448e-13 	iterations =  75 	Test passes:  True
